In [1]:
import os
import pickle
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from dtw import *
from tqdm import tqdm_notebook as tqdm

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
def visualize_labels(y, alt=False):
    color = 'cividis' if alt else 'plasma'
    
    idx = np.arange(len(y))

    fig, ax1 = plt.subplots()

    im = ax1.imshow(y[np.newaxis,:], cmap=color, aspect="auto")
    ax1.set_yticks([])
    fig.colorbar(im, ax=ax1)

    plt.tight_layout()
    plt.show()

In [3]:
def visualize_matrix(Y, alt=False):
    color = 'cividis' if alt else 'plasma'
    
    fig, ax1 = plt.subplots()
    
    im = ax1.imshow(Y, cmap=color, interpolation='nearest', aspect="auto")
    fig.colorbar(im, ax=ax1)
    
    plt.tight_layout()
    plt.show()

In [4]:
#task = 'task1_sandwich_openface'
task = 'task2_bart_openface'
#task = 'task3_jenga_openface'

model_path = './models/{0}/'.format(task)
path = './data/openface/{0}/'.format(task)
results_path = './data/final/{}_action_units.csv'.format(task)

## Pairs

In [5]:
num_paid_participants = 22
num_sona_participants = 17

task1_pairs = [('{0:02}01_sandwich_cut.csv'.format(i),'{0:02}02_sandwich_cut.csv'.format(i)) for i in range(1,num_paid_participants)]
task1_pairs += [('Sona{0:02}01_sandwich_cut.csv'.format(i),'Sona{0:02}02_sandwich_cut.csv'.format(i)) for i in range(1,num_sona_participants)]

task2_pairs = [('{0:02}01_bart_cut.csv'.format(i),'{0:02}02_bart_cut.csv'.format(i)) for i in range(1,num_paid_participants)]
task2_pairs += [('Sona{0:02}01_bart_cut.csv'.format(i),'Sona{0:02}02_bart_cut.csv'.format(i)) for i in range(1,num_sona_participants)]

task3_pairs = [('{0:02}01_jenga_cut.csv'.format(i),'{0:02}02_jenga_cut.csv'.format(i)) for i in range(1,num_paid_participants)]
task3_pairs += [('Sona{0:02}01_jenga_cut.csv'.format(i),'Sona{0:02}02_jenga_cut.csv'.format(i)) for i in range(1,num_sona_participants)]

pairs = {
    'task1_sandwich_openface': task1_pairs,
    'task2_bart_openface': task2_pairs,
    'task3_jenga_openface': task3_pairs
}

## Action Units

In [6]:
units = [1,2,4,5,7,9,10,12,14,15,17,20,23,25,26,45]
au_indicies = ['AU{:02}_r'.format(units[i]) for i in range(0, len(units))]

In [7]:
data = []
for i in tqdm(range(0,len(pairs[task]))):
    (p1, p2) = pairs[task][i]
    
    print(p1,p2)

    file1 = os.path.join(path,p1)
    file2 = os.path.join(path,p2)

    # load action units
    success1 = True
    try:
        df1 = pd.read_csv(file1, error_bad_lines=False, warn_bad_lines=True)   
        aus1 = df1[au_indicies]
    except:
        success1 = False

    if success1:
        nans = df1[df1[au_indicies].isnull().any(axis=1)]
        if not nans.empty:
            dropList = nans.index.tolist()
            for r in dropList:
                warnings.warn('Dropping row due to NaN, {0}'.format(r))
            aus1.drop(dropList, inplace=True)

        del df1
            
    success2 = True
    try:
        df2 = pd.read_csv(file2, error_bad_lines=False, warn_bad_lines=True)
        aus2 = df2[au_indicies]
    except:
        success2 = False

    if success2:
        nans = df2[df2[au_indicies].isnull().any(axis=1)]
        if not nans.empty:
            dropList = nans.index.tolist()
            for r in dropList:
                warnings.warn('Dropping row due to NaN, {0}'.format(r))
            aus2.drop(dropList, inplace=True)
            
        del df2

    if success1 and success2:    
        # dtw
        print('running dtw')
        alignment = dtw(aus2, aus1, keep_internals=True, 
                    open_begin=True, open_end=True, 
                    step_pattern='asymmetric')

        # results
        ave_cost = np.mean(alignment.costMatrix)
        min_dist = alignment.distance
        nor_dist = alignment.normalizedDistance
        
        del alignment

        data.append([True,True,p1,p2,ave_cost,min_dist,nor_dist])
    else:
        data.append([success1,success2,p1,p2])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/37 [00:00<?, ?it/s]

0101_bart_cut.csv 0102_bart_cut.csv
running dtw
0201_bart_cut.csv 0202_bart_cut.csv
running dtw
0301_bart_cut.csv 0302_bart_cut.csv


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: UserWarning: Dropping row due to NaN, 9935
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


0401_bart_cut.csv 0402_bart_cut.csv
running dtw
0501_bart_cut.csv 0502_bart_cut.csv
running dtw
0601_bart_cut.csv 0602_bart_cut.csv
running dtw
0701_bart_cut.csv 0702_bart_cut.csv
running dtw
0801_bart_cut.csv 0802_bart_cut.csv
running dtw
0901_bart_cut.csv 0902_bart_cut.csv
1001_bart_cut.csv 1002_bart_cut.csv
running dtw
1101_bart_cut.csv 1102_bart_cut.csv
running dtw
1201_bart_cut.csv 1202_bart_cut.csv
running dtw
1301_bart_cut.csv 1302_bart_cut.csv
running dtw
1401_bart_cut.csv 1402_bart_cut.csv
running dtw
1501_bart_cut.csv 1502_bart_cut.csv
1601_bart_cut.csv 1602_bart_cut.csv
1701_bart_cut.csv 1702_bart_cut.csv
1801_bart_cut.csv 1802_bart_cut.csv
1901_bart_cut.csv 1902_bart_cut.csv
2001_bart_cut.csv 2002_bart_cut.csv
2101_bart_cut.csv 2102_bart_cut.csv
Sona0101_bart_cut.csv Sona0102_bart_cut.csv
running dtw
Sona0201_bart_cut.csv Sona0202_bart_cut.csv
running dtw
Sona0301_bart_cut.csv Sona0302_bart_cut.csv
running dtw
Sona0401_bart_cut.csv Sona0402_bart_cut.csv
running dtw
Sona0501

In [9]:
outDf = pd.DataFrame(data, columns=['found1','found2','file1','file2','aveCost','minDist','norDist'])
outDf.to_csv(results_path, index=False)

In [10]:
outDf.head()

,found1,found2,file1,file2,aveCost,minDist,norDist
0,True,True,0101_bart_cut.csv,0102_bart_cut.csv,14384.703727,27272.597885,2.600610
1,True,True,0201_bart_cut.csv,0202_bart_cut.csv,8529.185129,16222.931391,1.236881
2,False,True,0301_bart_cut.csv,0302_bart_cut.csv,NaN,NaN,NaN
3,True,True,0401_bart_cut.csv,0402_bart_cut.csv,15219.460291,24505.566581,1.210391
4,True,True,0501_bart_cut.csv,0502_bart_cut.csv,17834.248494,29977.387346,0.999446
